<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Tutorials/Natural%20Language%20Processing/PyTorch%20Sentimental%20Analysis/First_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F

In [2]:
SEED = 1234
torch.manual_seed(SEED)

In [0]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype=torch.float)

In [4]:
from torchtext import datasets
train_dataset , test_dataset = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.3MB/s]


In [9]:
len(train_dataset), len(test_dataset)

(25000, 25000)

In [0]:
import random

train_data, validation_data = train_dataset.split(random_state= random.seed(SEED))

In [14]:
len(train_data), len(validation_data)

(17500, 7500)

In [0]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [24]:
print(len(TEXT.vocab), len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(20))


25002 2
[('the', 202273), (',', 191724), ('.', 165509), ('and', 109826), ('a', 109039), ('of', 100566), ('to', 93435), ('is', 76237), ('in', 61513), ('I', 53946), ('it', 53635), ('that', 49166), ('"', 44415), ("'s", 42942), ('this', 42362), ('-', 37016), ('/><br', 35354), ('was', 35200), ('as', 30258), ('with', 29916)]


In [0]:
BATCH_SIZES= 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits( (train_data, validation_data, test_dataset ), batch_size=BATCH_SIZES, device=device)

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dims, embedded_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dims, embedded_dim)
        self.rnn = nn.RNN(embedded_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))
        

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
